In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1401209,2021-06-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1401210,2021-06-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1401211,2021-06-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1401212,2021-06-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
37047,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37049,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37051,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37053,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
37055,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
858600,2021-06-24,Arkansas,Arkansas,5001,2201,38.00,5000,Arkansas,AR,Arkansas,State,3017804
858602,2021-06-25,Arkansas,Arkansas,5001,2205,38.00,5000,Arkansas,AR,Arkansas,State,3017804
858604,2021-06-26,Arkansas,Arkansas,5001,2205,38.00,5000,Arkansas,AR,Arkansas,State,3017804
858606,2021-06-27,Arkansas,Arkansas,5001,2205,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1401209,2021-06-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1401210,2021-06-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1401211,2021-06-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1401212,2021-06-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-06-28') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
524,2021-06-28,Snohomish,Washington,40136,600.00,53061,WA,County,822083,4882.23,72.99
1046,2021-06-28,Cook,Illinois,556885,11041.00,17031,IL,County,5150233,10812.81,214.38
1567,2021-06-28,Orange,California,273427,5123.00,6059,CA,County,3175692,8610.00,161.32
2087,2021-06-28,Maricopa,Arizona,559345,10268.00,4013,AZ,County,4485414,12470.31,228.92
2607,2021-06-28,Los Angeles,California,1250227,24493.00,6037,CA,County,10039107,12453.57,243.98
...,...,...,...,...,...,...,...,...,...,...,...
1400301,2021-06-28,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1400560,2021-06-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1400788,2021-06-28,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1401012,2021-06-28,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
524,2021-06-28,Snohomish,Washington,40136,600.00,53061,WA,County,822083,4882.23,72.99,72.99,4882.23
1046,2021-06-28,Cook,Illinois,556885,11041.00,17031,IL,County,5150233,10812.81,214.38,214.38,10812.81
1567,2021-06-28,Orange,California,273427,5123.00,6059,CA,County,3175692,8610.00,161.32,161.32,8610.00
2087,2021-06-28,Maricopa,Arizona,559345,10268.00,4013,AZ,County,4485414,12470.31,228.92,228.92,12470.31
2607,2021-06-28,Los Angeles,California,1250227,24493.00,6037,CA,County,10039107,12453.57,243.98,243.98,12453.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400301,2021-06-28,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1400560,2021-06-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1400788,2021-06-28,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1401012,2021-06-28,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
897907,2021-06-28,Hale,Alabama,2272,78.00,1065,AL,County,14651,15507.47,532.39,532.39,15507.47,1
977627,2021-06-28,Clarke,Alabama,3532,61.00,1025,AL,County,23622,14952.16,258.23,258.23,14952.16,2
702081,2021-06-28,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3
622334,2021-06-28,Etowah,Alabama,14237,365.00,1055,AL,County,102268,13921.27,356.91,356.91,13921.27,4
522006,2021-06-28,Franklin,Alabama,4322,81.00,1059,AL,County,31362,13781.01,258.27,258.27,13781.01,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325535,2021-06-28,Platte,Wyoming,680,12.00,56031,WY,County,8393,8101.99,142.98,142.98,8101.99,19
856779,2021-06-28,Converse,Wyoming,1038,18.00,56009,WY,County,13822,7509.77,130.23,130.23,7509.77,20
1065964,2021-06-28,Lincoln,Wyoming,1423,14.00,56023,WY,County,19830,7176.00,70.60,70.60,7176.00,21
1187010,2021-06-28,Niobrara,Wyoming,163,2.00,56027,WY,County,2356,6918.51,84.89,84.89,6918.51,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
702081,2021-06-28,Lowndes,Alabama,1421,54.00,1085,AL,County,9726,14610.32,555.21,555.21,14610.32,3,1
897907,2021-06-28,Hale,Alabama,2272,78.00,1065,AL,County,14651,15507.47,532.39,532.39,15507.47,1,2
620490,2021-06-28,Clay,Alabama,1602,59.00,1027,AL,County,13235,12104.27,445.79,445.79,12104.27,20,3
298061,2021-06-28,Walker,Alabama,7362,281.00,1127,AL,County,63521,11589.87,442.37,442.37,11589.87,29,4
621873,2021-06-28,Dallas,Alabama,3613,158.00,1047,AL,County,37196,9713.41,424.78,424.78,9713.41,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065964,2021-06-28,Lincoln,Wyoming,1423,14.00,56023,WY,County,19830,7176.00,70.60,70.60,7176.00,21,19
755441,2021-06-28,Hot Springs,Wyoming,372,3.00,56017,WY,County,4413,8429.64,67.98,67.98,8429.64,17,20
1039296,2021-06-28,Uinta,Wyoming,2318,13.00,56041,WY,County,20226,11460.50,64.27,64.27,11460.50,5,21
296660,2021-06-28,Teton,Wyoming,3801,11.00,56039,WY,County,23464,16199.28,46.88,46.88,16199.28,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.99,4882.23,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.99,4882.23,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.99,4882.23,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.99,4882.23,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.99,4882.23,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397940,2021-06-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1397941,2021-06-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1397942,2021-06-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1397943,2021-06-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
894183,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,1.00
894184,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,0.00
894185,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,0.00
894186,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,0.00
894187,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15507.47,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083888,2021-06-24,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,158.23,6039.03,7,23,0.00
1083889,2021-06-25,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,158.23,6039.03,7,23,0.00
1083890,2021-06-26,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,158.23,6039.03,7,23,0.00
1083891,2021-06-27,Crook,Wyoming,458,12.00,56011,WY,County,7584,6039.03,158.23,158.23,6039.03,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
699269,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,1.00,0.00
699270,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
699271,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
699272,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
699273,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14610.32,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696964,2021-06-24,Albany,Wyoming,4607,17.00,56001,WY,County,38880,11849.28,43.72,43.72,11898.15,23,3,5.00,0.00
696965,2021-06-25,Albany,Wyoming,4615,17.00,56001,WY,County,38880,11869.86,43.72,43.72,11898.15,23,3,8.00,0.00
696966,2021-06-26,Albany,Wyoming,4615,17.00,56001,WY,County,38880,11869.86,43.72,43.72,11898.15,23,3,0.00,0.00
696967,2021-06-27,Albany,Wyoming,4615,17.00,56001,WY,County,38880,11869.86,43.72,43.72,11898.15,23,3,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-06-28') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
355937,2021-06-28,Imperial,California,28987,741.00,6025,CA,County,181215,15995.92,408.91,408.91,15995.92,1,2,17.00,0.00
2607,2021-06-28,Los Angeles,California,1250227,24493.00,6037,CA,County,10039107,12453.57,243.98,243.98,12453.57,2,5,256.00,3.00
177001,2021-06-28,San Bernardino,California,299672,4928.00,6071,CA,County,2180085,13745.89,226.05,226.05,13745.89,3,4,68.00,0.00
624130,2021-06-28,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
79037,2021-06-28,Stanislaus,California,63685,1074.00,6099,CA,County,550660,11565.21,195.04,195.04,11565.21,5,9,82.00,1.00
62855,2021-06-28,San Joaquin,California,74808,1440.00,6077,CA,County,762148,9815.42,188.94,188.94,9815.42,6,17,0.00,0.00
36567,2021-06-28,Riverside,California,301799,4627.00,6065,CA,County,2470546,12215.88,187.29,187.29,12215.88,7,7,0.00,0.00
98034,2021-06-28,Tulare,California,49823,850.00,6107,CA,County,466195,10687.16,182.33,182.33,10687.16,8,10,0.00,0.00
565473,2021-06-28,Merced,California,32360,481.00,6047,CA,County,277680,11653.70,173.22,173.22,11653.70,9,8,25.00,0.00
35609,2021-06-28,Fresno,California,103110,1727.00,6019,CA,County,999101,10320.28,172.86,172.86,10320.28,10,14,19.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1325086,2021-06-28,Lassen,California,5809,26.00,6035,CA,County,30573,19000.43,85.04,85.04,19000.43,35,1,0.00,1.00
355937,2021-06-28,Imperial,California,28987,741.00,6025,CA,County,181215,15995.92,408.91,408.91,15995.92,1,2,17.00,0.00
757220,2021-06-28,Kings,California,23236,247.00,6031,CA,County,152940,15192.89,161.50,161.50,15192.89,11,3,2.00,0.00
177001,2021-06-28,San Bernardino,California,299672,4928.00,6071,CA,County,2180085,13745.89,226.05,226.05,13745.89,3,4,68.00,0.00
2607,2021-06-28,Los Angeles,California,1250227,24493.00,6037,CA,County,10039107,12453.57,243.98,243.98,12453.57,2,5,256.00,3.00
222633,2021-06-28,Kern,California,111049,1406.00,6029,CA,County,900202,12336.01,156.19,156.19,12336.01,13,6,101.00,1.00
36567,2021-06-28,Riverside,California,301799,4627.00,6065,CA,County,2470546,12215.88,187.29,187.29,12215.88,7,7,0.00,0.00
565473,2021-06-28,Merced,California,32360,481.00,6047,CA,County,277680,11653.70,173.22,173.22,11653.70,9,8,25.00,0.00
79037,2021-06-28,Stanislaus,California,63685,1074.00,6099,CA,County,550660,11565.21,195.04,195.04,11565.21,5,9,82.00,1.00
98034,2021-06-28,Tulare,California,49823,850.00,6107,CA,County,466195,10687.16,182.33,182.33,10687.16,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
355937,2021-06-28,Imperial,California,28987,741.00,6025,CA,County,181215,15995.92,408.91,408.91,15995.92,1,2,17.00,0.00
2607,2021-06-28,Los Angeles,California,1250227,24493.00,6037,CA,County,10039107,12453.57,243.98,243.98,12453.57,2,5,256.00,3.00
177001,2021-06-28,San Bernardino,California,299672,4928.00,6071,CA,County,2180085,13745.89,226.05,226.05,13745.89,3,4,68.00,0.00
624130,2021-06-28,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
79037,2021-06-28,Stanislaus,California,63685,1074.00,6099,CA,County,550660,11565.21,195.04,195.04,11565.21,5,9,82.00,1.00
62855,2021-06-28,San Joaquin,California,74808,1440.00,6077,CA,County,762148,9815.42,188.94,188.94,9815.42,6,17,0.00,0.00
36567,2021-06-28,Riverside,California,301799,4627.00,6065,CA,County,2470546,12215.88,187.29,187.29,12215.88,7,7,0.00,0.00
98034,2021-06-28,Tulare,California,49823,850.00,6107,CA,County,466195,10687.16,182.33,182.33,10687.16,8,10,0.00,0.00
565473,2021-06-28,Merced,California,32360,481.00,6047,CA,County,277680,11653.70,173.22,173.22,11653.70,9,8,25.00,0.00
35609,2021-06-28,Fresno,California,103110,1727.00,6019,CA,County,999101,10320.28,172.86,172.86,10320.28,10,14,19.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.91,15995.92,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4672,06/24/21,Lassen,5799,24.00,18967.72,78.50,85.04,19000.43,35,1,0.00,0.00
4673,06/25/21,Lassen,5808,25.00,18997.15,81.77,85.04,19000.43,35,1,9.00,1.00
4674,06/26/21,Lassen,5809,25.00,19000.43,81.77,85.04,19000.43,35,1,1.00,0.00
4675,06/27/21,Lassen,5809,25.00,19000.43,81.77,85.04,19000.43,35,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.91,15995.92,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.91,15995.92,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4758,06/24/21,Fresno,103020,1724.00,10311.27,172.56,172.86,10320.28,10,14,24.00,0.00
4759,06/25/21,Fresno,103061,1727.00,10315.37,172.86,172.86,10320.28,10,14,41.00,3.00
4760,06/26/21,Fresno,103073,1727.00,10316.57,172.86,172.86,10320.28,10,14,12.00,0.00
4761,06/27/21,Fresno,103091,1727.00,10318.38,172.86,172.86,10320.28,10,14,18.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)